In [8]:
"""
This is a file to make classification for IWT memes
"""
import pandas as pd 
import torch
import torch.nn as nn
import json
import classifier_utilities as cu
import torch.nn.functional as F
import os
import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
from torch.optim.lr_scheduler import MultiStepLR
from skimage import io, transform
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from PIL import ImageFile
from torch import optim
from collections import Counter
import regex as re
import time
from sklearn.metrics import recall_score, precision_score, accuracy_score
import bcolz
import pickle
from pytesseract import image_to_string
import TwitterDataset as td
import MemeModel as mm
import train_validation_test_function as tvt
import csv
import matplotlib.pyplot as plt
import help as hp
ImageFile.LOAD_TRUNCATED_IMAGES = True
from collections import OrderedDict
import parameters as 
threshold =0.31037354

## Loading the pretrained word embedding matrix and backbone for model initiation

glove_dir = '/data/yuhao/web_sci/meme_classifier/glove.6B/'
vectors = bcolz.open(glove_dir + '6B.50.dat')[:]
words = pickle.load(open(glove_dir + '6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(glove_dir + '6B.50_idx.pkl', 'rb'))
matrix = torch.FloatTensor(vectors)
resnet = models.resnet50(pretrained = True) # load pretrained the resnet50 model
backbone = nn.Sequential(*list(resnet.children())[:-1]) 
mean_1 = 0.579662
mean_2 = 0.5555058
mean_3 = 0.5413896
std_1 = 0.3494197
std_2 = 0.3469673
std_3 = 0.35115704

composed = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean=[mean_1, mean_2, mean_3],std=[std_1, std_2, std_3])])

composed_vali = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[mean_1, mean_2, mean_3],std=[std_1, std_2, std_3])])

test_root = 

# load test_dataset
dict_1 =json.load(open('/data/yuhao/web_sci/meme_classifier/test/dic1.json','rb'))
test_dataset = td.TwitterDataset3(test_root,word2idx,dict_1,cu.max_len,transform = composed_vali)
model = mm.Meme_classifier2(backbone,matrix,cu.hidden_size2,cu.hidden_size,cu.batch_size)
loss = nn.BCELoss()
test_loss_p = 1000000
index = 0

def test(model, loss, gpu,dataset,threshold):
    """
    This is the function for evaluating the model
    Input: model: evaluation model
           loader: dataloader
           criterion: critertion for loss
           gpu: using gpu or not (binary)
           dataset: dataset for evaluation
    Output: Directory_IWTmeme: 
            Directory_nonIWTmeme:
    """
    dataloader = DataLoader(dataset, batch_size = cu.batch_size, shuffle = True, num_workers = 4)

    model.eval()
    current_loss = 0
    target = []
    output = []
    preds = []
    name = []
    for (image_,text_ ,name_) in dataloader:
        image_,text_ = image_.cuda(),text_.type(torch.LongTensor).cuda()
        model = model.cuda() # put data into GPU
        output_ = model.forward(image_,text_).type(torch.FloatTensor)
        preds.extend(1-output_.detach().cpu().numpy())
        name.extend(list(name_))
    preds = [True if i > threshold else False for i in preds ]
    IWTmeme_name = [i for index,i in enumerate(name) if preds[index]]
    nonIWTmeme_name = [i for index,i in enumerate(name) if not preds[index]]
    return IWTmeme_name, nonIWTmeme_name


gpu = True # use GPU or not
model_dir = './checkpoint/model.pth'
checkpoint = torch.load(model_dir)
model.load_state_dict(checkpoint['model_state'])
IWTmeme_name, nonIWTmeme_name = test(model,loss,gpu,test_dataset,threshold)



In [7]:
os.